<a href="https://colab.research.google.com/github/theysee/sberbank_reboot/blob/master/%D0%9F%D0%BE%D1%82%D0%BE%D0%BA_3_%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_%D0%BA_%D0%B7%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D1%8E_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Практика



*   [Презентация](https://docs.google.com/presentation/d/100xFwSILlxsoaAj98FW5ztneo03fRyob-33IKOvpJqM/edit?usp=sharing)
*   [master](https://colab.research.google.com/drive/1ZmBW1QzhWjPQGs1E3oFDvaygKZZAQ6CP)



In [1]:
!pip install PyMySQL

     |████████████████████████████████| 51kB 2.5MB/s 


In [2]:
!pip install pandas

In [0]:
import pymysql
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline


# доступны student-ы от 0 до 49
# в этот раз важно взять своего student-а. Если в коде ниже увидите, что в вашей базе уже есть таблицы, то выберите другого пользователя.
# НЕ ЗАБУДЬТЕ ВЫПОЛНИТЬ КОД ЭТОЙ КЛЕТКИ



DB_HOST = '89.223.95.235'
DB_USER = 'student_1'
DB_USER_PASSWORD = 'student_1'
DB_NAME = 'student_1'

In [4]:
conn = pymysql.connect(DB_HOST, DB_USER, DB_USER_PASSWORD, DB_NAME)
pd.read_sql_query("show tables", conn)

,Tables_in_student_1
0,adv_events
1,adv_glossary
2,advertisment


### Шаг 1
Сгенерируем данные по показам, кликам, и деньгам по 5 объявлениям с названиями Advertising_0, Advertising_1, ...

Просто выполните две клетки ниже

In [5]:
adv_sample = 'Advertising_{}'
advertisings = []
for i in range(5):
  advertisings.append(adv_sample.format(i))
advertisings

['Advertising_0',
 'Advertising_1',
 'Advertising_2',
 'Advertising_3',
 'Advertising_4']

In [6]:
from datetime import datetime, timedelta, date
DATE_FORMAT = '%Y-%m-%d'
start_date = date(2019, 8, 1)
data = {}
for adv_id in range(1, len(advertisings)+1):
  day_data = []
  data[adv_id] = []
  for idx, events_number in enumerate(np.random.random_integers(50, 100, 30)):
    events = np.zeros(events_number)
    events[np.random.rand(events_number) > 0.5] = 1
    money = np.round(events * np.random.rand(events_number)*10 + np.random.rand(events_number), 2)
    dt = start_date + timedelta(days=idx)
    dates = [dt.strftime(DATE_FORMAT) for j in range(events_number)]
    day_data = list(zip(dates, events, money))
    data[adv_id] += day_data
data

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. Please call randint(50, 100 + 1) instead
  


{1: [('2019-08-01', 0.0, 0.65),
  ('2019-08-01', 0.0, 0.06),
  ('2019-08-01', 1.0, 7.75),
  ('2019-08-01', 0.0, 0.69),
  ('2019-08-01', 1.0, 2.32),
  ('2019-08-01', 0.0, 0.44),
  ('2019-08-01', 0.0, 0.5),
  ('2019-08-01', 1.0, 3.87),
  ('2019-08-01', 0.0, 0.45),
  ('2019-08-01', 0.0, 0.28),
  ('2019-08-01', 1.0, 6.65),
  ('2019-08-01', 0.0, 0.27),
  ('2019-08-01', 0.0, 0.29),
  ('2019-08-01', 1.0, 9.11),
  ('2019-08-01', 1.0, 1.01),
  ('2019-08-01', 1.0, 7.69),
  ('2019-08-01', 1.0, 7.68),
  ('2019-08-01', 1.0, 1.97),
  ('2019-08-01', 1.0, 6.25),
  ('2019-08-01', 0.0, 0.96),
  ('2019-08-01', 0.0, 0.64),
  ('2019-08-01', 1.0, 7.45),
  ('2019-08-01', 1.0, 1.52),
  ('2019-08-01', 1.0, 3.25),
  ('2019-08-01', 1.0, 9.52),
  ('2019-08-01', 0.0, 0.28),
  ('2019-08-01', 0.0, 0.08),
  ('2019-08-01', 1.0, 4.85),
  ('2019-08-01', 1.0, 7.68),
  ('2019-08-01', 1.0, 2.39),
  ('2019-08-01', 0.0, 0.05),
  ('2019-08-01', 0.0, 0.82),
  ('2019-08-01', 0.0, 0.39),
  ('2019-08-01', 1.0, 3.2),
  ('2019-08-0

In [7]:
data.keys()

dict_keys([1, 2, 3, 4, 5])

Теперь в массиве advertisings у нас лежат названия объявлений, а в словаре data ключом является идентификатор объявления в нашей будущей базе данных (при использовании AUTO_INCREMENT PRIMARY KEY идентификаторы будут от 1 до 5 соответственно). 

Значениями в словаре data являются массивы tuple-ов в которых по порядку записана дата события, тип события, а также кол-во денег, которое стоило это событие. 

*   0 это показ
*   1 это клик (если был клик, то был и показ)
*   кол-во денег указано в рублях



In [8]:
data[1][0][2]

0.65

### Шаг 2
Создадим структуру таблиц.
Нам понадобится создать две таблицы. В одной будут храниться названия объявлений с их уникальным идентификатором, а во второй будут храниться события, относящиеся к этим объявлениям. При этом вторая таблица связана с первой таблицей первичным ключом.

In [0]:
# выполните, чтобы получить объект соединения
conn = pymysql.connect(DB_HOST, DB_USER, DB_USER_PASSWORD, DB_NAME) # создаём соединение

In [0]:
# выполните эту клетку если захотите удалить ошибочно созданную таблицу adv_glossary
query = "DROP TABLE IF EXISTS adv_glossary"

conn.cursor().execute(query) # выполняем запрос
conn.commit() # сохраняем изменения

In [0]:
# выполните эту клетку если захотите удалить ошибочно созданную таблицу adv_events
query = "DROP TABLE IF EXISTS adv_events"


conn.cursor().execute(query) # выполняем запрос
conn.commit() # сохраняем изменения

In [0]:
# напишите запрос для создания таблицы с объявлениями

In [0]:
# напишите запрос для создания таблицы с событиями по объявлениям

In [0]:
# а также добавьте ограничение целостности на идентификатор объявления

### Шаг 3
Теперь, когда таблицы созданы, наполните их данными. Это может занять некоторое время, подождите, выпейте чаю;). Заметьте, если внешний ключ создан правильно, то база не даст вписать данные с несуществующими связями. 
С какой базы надо начать наполнение?

In [0]:
# ваш код тут

### Шаг 4
Проверьте, что данные действительно появились в таблицах

In [0]:
# ваш код тут

### Шаг 5
Теперь для каждого объявления посчитайте [CTR](https://ru.wikipedia.org/wiki/CTR_(%D0%98%D0%BD%D1%82%D0%B5%D1%80%D0%BD%D0%B5%D1%82)#targetText=CTR%20(%D1%81%D0%B8%D0%BD%D0%BE%D0%BD%D0%B8%D0%BC%20%E2%80%94%20%D0%BA%D0%BB%D0%B8%D0%BA%D0%B0%D0%B1%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D1%8C%2C%20%D0%BE%D1%82,%D1%87%D0%B8%D1%81%D0%BB%D1%83%20%D0%BF%D0%BE%D0%BA%D0%B0%D0%B7%D0%BE%D0%B2%2C%20%D0%B8%D0%B7%D0%BC%D0%B5%D1%80%D1%8F%D0%B5%D1%82%D1%81%D1%8F%20%D0%B2%20%D0%BF%D1%80%D0%BE%D1%86%D0%B5%D0%BD%D1%82%D0%B0%D1%85.&targetText=%D0%9D%D0%B0%D0%BF%D1%80%D0%B8%D0%BC%D0%B5%D1%80%3A%20%D1%80%D0%B5%D0%BA%D0%BB%D0%B0%D0%BC%D0%BD%D1%8B%D0%B9%20%D0%B1%D0%BB%D0%BE%D0%BA%20%D0%B1%D1%8B%D0%BB%20%D0%BF%D0%BE%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD,%D0%BD%D0%B0%20%D0%BD%D0%B5%D0%B3%D0%BE%20%D0%BA%D0%BB%D0%B8%D0%BA%D0%BD%D1%83%D0%BB%D0%B8%202%20%D1%80%D0%B0%D0%B7%D0%B0.) и [CPM](http://promo-sa.ru/seo-terms/cpm) за каждый день с помощью sql запроса, и выведите результат в pandas.

В результирующем датафрейме должно быть отражено название объявления, дата, CTR за день, CPM за день.

In [17]:
# data = pd.read_sql_query("", conn) # считаем данные в переменную
# Всё сводится к тому ,чтобы посчитать кол-во кликов и кол-во показов. 
# Подсказка: используйте конструкцию CASE WHEN и агрегатную функцию SUM
pd.read_sql_query("", conn)

DatabaseError: ignored

### Шаг 6
Но на таблицу не так интересно смотреть как на графики...

Для каждого объявления постройте дневные графики CTR и CPM.

На всякий случай проследите, чтобы данные были нарисованы по возрастанию даты. Для этого надо либо отсортировать датафрейм по дате, либо проследить, что тип данных у это поля в pandas что-то типа date. Чтобы посмотреть типы полей в pandas используйте атрибут датафрейма .dtypes